In [71]:
#     This program is free software: you can redistribute it and/or modify
#     it under the terms of the GNU General Public License as published by
#     the Free Software Foundation, either version 3 of the License, or
#     (at your option) any later version.
#
#     This program is distributed in the hope that it will be useful,
#     but WITHOUT ANY WARRANTY; without even the implied warranty of
#     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#     GNU General Public License for more details.
#
#     You should have received a copy of the GNU General Public License
#     along with this program.  If not, see <https://www.gnu.org/licenses/>.

#     Written by Charalambos (Charis) Poullis - www.poullis.org

In [72]:
#Import all necessary libraries
import torch
import torchvision
import numpy as np
#progress bar functionality
import tqdm
#plotting etc
import matplotlib

import matplotlib.pyplot as plt
#random number generator
import random

# There is a weird crash in Python 3.9 causing the kernel to restart when using matplotlib
# To test if you have the same issue uncomment the following command. If it works, then you don't need the subsequent 2 lines (import os, KMP)
#plt.subplot()
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [73]:
#Set the path for the datasets
DATASET_PATH = "../data"

#MNIST image size 1x28x28
CHANNELS = 1
WIDTH = 28
HEIGHT = 28

#Download the training dataset: CIFAR10
training_dataset = torchvision.datasets.MNIST(root=DATASET_PATH,
                                                train=True,
                                                download=True,
                                                transform=torchvision.transforms.ToTensor())
print('Training dataset:', training_dataset)

#Download the testing dataset: CIFAR10
testing_dataset = torchvision.datasets.MNIST(root=DATASET_PATH,
                                               train=False,
                                               download=True,
                                               transform=torchvision.transforms.ToTensor())
print('Testing dataset:', testing_dataset)  

#Create a list with user-friendly names for each label
labels=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

Training dataset: Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: ToTensor()
Testing dataset: Dataset MNIST
    Number of datapoints: 10000
    Root location: ../data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [74]:
#This function takes a CIFAR dataset and splits it into two numpy arrays containing
#the images and labels, respectively

def splitDataset(dataset, channels, width, height):
    #Divide the datasets into images and labels
    x, y = dataset[0]
    
    shape = [(i, *j) for i, j in [(len(dataset), x.shape)]][0]
    images = np.zeros(shape)
    labels = np.zeros((len(dataset), 1))
    
    for i in range(len(dataset)):
        x,y = dataset[i]
        images[i] = x
        labels[i] = y
        
    images = np.reshape(images, (len(dataset), channels, width, height))
    labels = np.reshape(labels, (len(dataset), 1))
    return images, labels

In [75]:
#Divide the training datasets into images and labels; this is needed later to randomly index from the arrays
training_images, training_labels = splitDataset(training_dataset, CHANNELS, WIDTH, HEIGHT)
#Do the same for the testing dataset
testing_images, testing_labels = splitDataset(testing_dataset, CHANNELS, WIDTH, HEIGHT)   

print(f'Training images: {len(training_images)}, Training labels: {len(training_labels)}')
print(f'Testing images: {len(testing_images)}, Testing labels: {len(testing_labels)}')

Training images: 60000, Training labels: 60000
Testing images: 10000, Testing labels: 10000


In [76]:
#Define the autoencoder; This is a simple autoencoder with 2 convolutional layers in the encoder, and 2 convolutional layers in the decoder
#Put simply, an autoencoder learns to reconstruct the input
class MyAutoencoder (torch.nn.Module):
    def __init__(self, channels, width, height):
        super(MyAutoencoder, self).__init__()

        #The patch size
        self.channels = channels
        self.width = width
        self.height = height

        #The latent dimensions
        self.latent_space_dims = 4

        #Define a unit Gaussian distribution to be used for sampling
        self.unit_gaussian_distribution = torch.distributions.normal.Normal(loc=torch.tensor(0.0).to(device),
                                                                            scale=torch.tensor(1.0).to(device))

        #The KL divergence
        self.KL_div = torch.tensor(0.0)

        #Encoder
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(in_features=self.channels*self.width*self.height, out_features=512),
            torch.nn.ReLU()
        )

        #μ
        self.mu = torch.nn.Linear(in_features=512, out_features=self.latent_space_dims)
        #σ
        self.sigma = torch.nn.Linear(in_features=512, out_features=self.latent_space_dims)

        #Decoder
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(in_features=self.latent_space_dims, out_features=512),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=512, out_features=self.channels*self.width*self.height),

            #At this point the range of the output value is unbound. Let's use Maps the value to the range [0,1] which is expected for colors
            torch.nn.Sigmoid()
        )
        return

    def getKLdiv(self):
        return self.KL_div
    
    def forward(self, x):
        x = torch.reshape(x, (-1, self.channels*self.width*self.height))
        #Feed-forward through the encoder
        result = self.encoder(x)
        
        #Sample z using the Gaussian distribution
        mu = self.mu(result)
        sigma = torch.exp(self.sigma(result)) #self.sigma(.) can return negative values. We add the exponent to keep the values positive. Otherwise, what is the meaning of having negative standard deviation σ?
        z = mu + sigma*self.unit_gaussian_distribution.sample(mu.shape)

        #Feed-forward through the decoder
        result = self.decoder(z)
        result = torch.reshape(result, (-1, self.channels, self.width, self.height))
        #Calculate the closed form KL divergence between 2 Gaussians; you want the learnt (μ,σ) to resemble a zero-mean Gaussian distribution with standard deviation of 1.0
        self.KL_div = torch.sum(-torch.log(sigma) + mu**2 + sigma**2 - 0.5)

        return result

In [77]:
#We'll use GPU is available; check if a GPU is available
print(f'GPU devices found: {torch.cuda.device_count()}')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

GPU devices found: 1
Device: cuda


In [78]:
#Create an instance of the model and print; it should display all the layers and their related information
model = MyAutoencoder(CHANNELS, WIDTH, HEIGHT).to(device)
print(model)

MyAutoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
  )
  (mu): Linear(in_features=512, out_features=4, bias=True)
  (sigma): Linear(in_features=512, out_features=4, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=4, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=784, bias=True)
    (3): Sigmoid()
  )
)


In [79]:
#SANITY CHECK: Feed forward an image and check for errors
#Pull the first image from the training images, and convert to a float tensor
img = torch.from_numpy(training_images[0]).float()
print(img.shape)

#The input to the autoencoder is [batch_size, 3, 32, 32]; convert the image's shape [3,32,32] to [1,3,32,32] and push to device
img = torch.reshape(img, (1, CHANNELS, WIDTH, HEIGHT)).to(device)

#feed forward the image to the network and make sure it works with no errors
model.forward(img)

torch.Size([1, 28, 28])


tensor([[[[0.4821, 0.5466, 0.4896, 0.5513, 0.4662, 0.4868, 0.4518, 0.4347,
           0.5025, 0.4456, 0.4709, 0.5344, 0.4508, 0.5570, 0.4362, 0.4423,
           0.5267, 0.3931, 0.5880, 0.4506, 0.4717, 0.4755, 0.5651, 0.5993,
           0.5179, 0.5053, 0.4864, 0.4874],
          [0.4565, 0.4381, 0.5693, 0.4857, 0.6260, 0.5566, 0.5613, 0.5138,
           0.5568, 0.5430, 0.4678, 0.4952, 0.4597, 0.4990, 0.6136, 0.5007,
           0.4590, 0.4603, 0.4837, 0.4026, 0.5809, 0.4124, 0.4755, 0.5652,
           0.4652, 0.3704, 0.4569, 0.5102],
          [0.4178, 0.4549, 0.5025, 0.5633, 0.4964, 0.6204, 0.5084, 0.6110,
           0.4652, 0.5645, 0.4421, 0.5307, 0.4126, 0.3894, 0.4673, 0.4064,
           0.4972, 0.5055, 0.4818, 0.4213, 0.5509, 0.5902, 0.5128, 0.5050,
           0.5613, 0.4601, 0.4579, 0.4835],
          [0.4623, 0.5467, 0.4608, 0.5324, 0.5064, 0.5748, 0.4349, 0.5486,
           0.5439, 0.5672, 0.5554, 0.4483, 0.5860, 0.4919, 0.5381, 0.5443,
           0.5338, 0.4559, 0.5510, 0.5498, 

In [80]:
#Switch to 0; otherwise it will load the saved weights
if 0:
    model.load_state_dict(torch.load("trained_weights_linear_MNIST.save"))
    model.eval()

In [ ]:
#TRAINING

#define the hyperparameters
BATCH_SIZE = 100
EPOCHS=200
LEARNING_RATE=0.00001


#Create an optimizer and pass the model's parameters
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

#Define a loss function; we'll pick mean squared error since we are reconstructing the original
loss_function = torch.nn.MSELoss(reduction='sum')

#training image indices; [0, 1, 2, 3 ....., 49999]
training_dataset_indices = torch.arange(0, len(training_dataset))

#keep track of the losses for plotting at the end
losses = list()

#Create a progress bar the size of the number of epochs
status_bar = tqdm.tqdm(range(EPOCHS), ncols=100, desc='loss: ')
for i in status_bar:
    #At each iteration, perform a permutation of the training_dataset_indices; this ensures that we will use all images and avoid duplication
    random_permutation = torch.randperm(training_dataset_indices.size()[0])

    #Get BATCH_SIZE number of training images at each epoch; because we sample from an array there will be no duplicates
    loss = None
    for b in range(0,training_dataset_indices.size()[0], BATCH_SIZE):
        #Zero out the gradients
        optimizer.zero_grad()

        #Get the indices of the training images in this batch
        batch_indices = random_permutation[b:b+BATCH_SIZE]
        #we only need to get the images; the labels are irrelevant in the context of an autoencoder
        batch_images = training_images[batch_indices]
        #convert to float tensors and push to device
        batch_images = torch.from_numpy(batch_images).float().to(device)

        #feed forward through the network's layers and get the reconstruction
        reconstructions = model.forward(batch_images)
        
        #Calculate the MSE between the reconstructions and the input images plus the KL divergence of the learnt distribution from a zero-mean Gaussian of standard deviation of 1.0; the optimal value for this loss is 0
        loss = loss_function(reconstructions, batch_images) + model.getKLdiv()

        #keep track of the losses per batch, per epoch it to the list; used for plotting later on
        losses.append(loss.item())
        
        #backpropagate gradients
        loss.backward()
        #single gradient descent step; update all network parameters based on the gradient
        optimizer.step()
    
    #After every epoch update the loss function
    status_bar.set_description(f'loss: {loss.item():.8f}')

#Output the final loss on the training dataset
print(f'Final loss on training: {loss.item():.8f}')

loss: 3618.47753906:  77%|██████████████████████████████▊         | 154/200 [03:51<01:08,  1.48s/it]

In [ ]:
#Switch to 0, otherwise it will overwrite the saved weights
if 1:
    torch.save(model.state_dict(), "trained_weights_linear_MNIST.save")

In [ ]:
#Plot the loss as a function of epochs
fig, ax = plt.subplots(1,1)
ax.plot(list(range(EPOCHS*int(len(training_dataset)/BATCH_SIZE))), np.array(losses))
ax.set_title('Loss on training dataset')
ax.set_xlabel('epoch')
ax.set_ylabel('MSE')
plt.show()
fig.savefig("vae_loss_linear_MNIST.png")

In [ ]:
with torch.no_grad():
    #Calculate MSE on the testing dataset; convert to float tensors and push to device; I'm passing all the images together since it fits in the GPU
    testing_images_ = torch.from_numpy(testing_images[:BATCH_SIZE]).float().reshape( (BATCH_SIZE, CHANNELS, WIDTH, HEIGHT) ).to(device)
    #get the reconstructions
    reconstructions = model.forward(testing_images_)
    testing_loss = loss_function(reconstructions, testing_images_).item()
    #Output the MSE loss
    print(f'MSE loss on testing: {testing_loss:.8f}')

In [ ]:
#Plot a couple of examples
cols, rows = 2, 10
image_counter = 1
figure = plt.figure(figsize=(3*cols, 3*rows))
for i in range(10):
    #get a random index
    rand_index = random.randint(0, len(testing_images_))
    #get the original image, pull back to cpu and convert to numpy
    original_img = testing_images_[rand_index].permute(1,2,0).cpu().detach().numpy()
    #get the reconstructed image, pull back to cpu and convert to numpy
    reconstructed_img = reconstructions[rand_index].permute(1,2,0).cpu().detach().numpy()
    
    #Add a figure for the original image
    figure.add_subplot(rows, cols, image_counter)
    plt.axis("off")
    plt.imshow(original_img)
    image_counter+=1

    #Add a figure for the reconstructed image
    figure.add_subplot(rows, cols, image_counter)
    plt.axis("off")
    plt.imshow(reconstructed_img)
    image_counter+=1

plt.show()
figure.savefig("vae_reconstruction_linear_MNIST.png")